<a href="https://colab.research.google.com/github/iued-uni-heidelberg/corpusdev/blob/main/bibliography_dataset_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content
!pwd

/content
/content


In [1]:
!git clone https://git.hab.de/beyer/vd17-dump/

Cloning into 'vd17-dump'...
remote: Enumerating objects: 4739, done.
remote: Counting objects: 100% (2862/2862), done.
remote: Compressing objects: 100% (1210/1210), done.
Receiving objects: 100% (4739/4739), 646.04 MiB | 13.88 MiB/s, done.
remote: Total 4739 (delta 2202), reused 2311 (delta 1652), pack-reused 1877
Resolving deltas: 100% (4033/4033), done.
Updating files: 100% (309/309), done.


In [6]:
!git clone https://github.com/hbeyer/pylib

Cloning into 'pylib'...
remote: Enumerating objects: 688, done.
remote: Counting objects: 100% (264/264), done.
remote: Compressing objects: 100% (189/189), done.
remote: Total 688 (delta 179), reused 158 (delta 75), pack-reused 424
Receiving objects: 100% (688/688), 397.32 KiB | 3.45 MiB/s, done.
Resolving deltas: 100% (444/444), done.


In [ ]:
!echo $PYTHONPATH

"$/env/python"


In [7]:
!echo $PATH

"/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/content/pylib/lib"


In [3]:
%env PATH="/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/content/pylib/lib"

env: PATH="/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/content/pylib/lib"


In [ ]:
%env PYTHONPATH="$/env/python"

env: PYTHONPATH="$/env/python"


In [16]:
%cd /content/pylib/
!pwd

/content/pylib
/content/pylib


In [12]:
from lib import bookwheel as bw

In [17]:
import logging
import pickle
from lib import sru
from lib import isil
from lib import xmlreader as xr
from lib import pica

In [ ]:
%env PYTHONPATH="$/env/python:/content/pylib/lib/:/content/pylib/lib/bookwheel.py"
# !PYTHONPATH=. ./comet/cli/score.py

env: PYTHONPATH="$/env/python:/content/pylib/lib/:/content/pylib/lib/bookwheel.py"


In [15]:
# from lib import bookwheel as bw
cat = bw.Catalogue
sec = cat.get_section(2589)
print(sec)


{'start': 2511, 'end': 2738, 'group': 'Libri Varii', 'dateBegin': '1634', 'year': 1634, 'writer': 'Herzog August'}


In [18]:
%cd /content/
!pwd

/content
/content


In [ ]:
# import logging
# import pickle
# from lib import sru
# from lib import isil
# from lib import xmlreader as xr
# from lib import pica

logging.basicConfig(level=logging.ERROR)

# Festlegen der Speicherpfade und der Datensätze pro JSON-Datei
# source_folder = "{Ordner mit den PICAXML-Dateien}/"
source_folder = "/content/xmlbibliography/"
# source_folder_ac = "{Ordner mit den PICAXML-Dateien für Gesamtaufnahmen mehrbändiger Werke (Ac-Sätze)}/"
source_folder_ac = "/content/xmlbibliography/"
# target_folder = "{Ordner zum Speichern der JSON-Dateien}/"
target_folder = "/content/jsonbibliography/"
size = 1000
limit = 350000

# Laden der Ac-Sätze und Extrahieren der Gattungsbegriffe
req = sru.Request_VD17()
num = req.prepare("pica.bbg=Ac*")
print(req.url)
print(req.numFound)
req.download(source_folder_ac)

res = {}
reader = xr.DownloadReader(source_folder_ac, "record", "info:srw/schema/5/picaXML-v1.0")

for count, node in enumerate(reader):
    rec = pica.RecordVD17(node)
    gatt = [gat for gat in rec.gatt]
    if gatt == []:
        continue
    res[rec.ppn] = gatt
    if count > 100000:
        break

with open('gattungen-ac', 'wb') as file:
    pickle.dump(res, file)

# Download der PICA-XML-Daten
req = sru.Request_VD17()
num = req.prepare("pica.bbg=(Aa* or Af*)")
print(req.url)
print(req.numFound)
req.download(source_folder)

# Auslesen und Abspeichern in JSON
with open('gattungen-ac','rb') as file:
    gatt_ac = pickle.load(file)

reader = xr.DownloadReader(source_folder, "record", "info:srw/schema/5/picaXML-v1.0")

content = []
fnn = []
setn = 1
count = 0

for no, node in enumerate(reader):
    rec = pica.RecordVD17(node)
    if rec.get_rec_type() in ["Teilband", "Teilband mit eigenem Titel"]:
        try:
            rec.gatt = gatt_ac[rec.ppn_sup]
        except:
            logging.info(f"Keine Gattungsbegriffe bei PPN {rec.ppn_sup}")
    content.append(rec)
    count += 1
    if count >= size:
        recl = pica.RecordList(content)
        fn = f"vd17-{str(setn).zfill(3)}"
        recl.to_json(target_folder + fn)
        content = []
        setn += 1
        count = 0
    if no > limit:
        break
if content != []:
    recl = pica.RecordList(content)
    fn = f"vd17-{str(setn).zfill(3)}"
    recl.to_json(target_folder + fn)


http://sru.k10plus.de/vd17?version=2.0&operation=searchRetrieve&query=pica.bbg%3DAc%2A&maximumRecords=500&startRecord=1&recordSchema=picaxml
5088
Download 1 erledigt
Download 2 erledigt
Download 3 erledigt
Download 4 erledigt
Download 5 erledigt
Download 6 erledigt
Download 7 erledigt
Download 8 erledigt
Download 9 erledigt
Download 10 erledigt
Download 11 erledigt


ERROR:root:Keine Datensätze


http://sru.k10plus.de/vd17?version=2.0&operation=searchRetrieve&query=pica.bbg%3D%28Aa%2A%20or%20Af%2A%29&maximumRecords=500&startRecord=1&recordSchema=picaxml
307241
Download 1 erledigt
Download 2 erledigt
Download 3 erledigt
Download 4 erledigt
Download 5 erledigt
Download 6 erledigt
Download 7 erledigt
Download 8 erledigt
Download 9 erledigt
Download 10 erledigt
Download 11 erledigt
Download 12 erledigt
Download 13 erledigt
Download 14 erledigt
Download 15 erledigt
Download 16 erledigt
Download 17 erledigt
Download 18 erledigt
Download 19 erledigt
Download 20 erledigt
Download 21 erledigt
Download 22 erledigt
Download 23 erledigt
Download 24 erledigt
Download 25 erledigt
Download 26 erledigt
Download 27 erledigt
Download 28 erledigt
Download 29 erledigt
Download 30 erledigt
Download 31 erledigt
Download 32 erledigt
Download 33 erledigt
Download 34 erledigt
Download 35 erledigt
Download 36 erledigt
Download 37 erledigt
Download 38 erledigt


In [19]:
!mkdir xmlbibliography

In [24]:
!mkdir jsonbibliography

In [20]:
%cd xmlbibliography/

/content/xmlbibliography


In [ ]:
!wget http://sru.k10plus.de/vd17?version=2.0&operation=searchRetrieve&query=pica.bbg=%28Aa*%20or%20Af*%29&maximumRecords=500&startRecord=1&recordSchema=picaxml

In [22]:
!mv vd17?version=2.0 vd17_500.xml

In [23]:
%cd /content/

/content
